Hunter S. DiCicco

with Dr. Dongwon Lee, Ph.D.

# Data Collection: Limited Web Scraping
for *Promotional* and *Misreporting* Content

## TODO:
* Rectify Promotional Content/Native Ads Class Balance
* Vectorize Human Trial Test Set

### Setup and Imports

In [1]:
import sys
import re
import os
import sys
import requests as r
import itertools as it

import numpy as np
import pandas as pd

from bs4 import BeautifulSoup

os.chdir('../')
import feature_extraction as fe

In [2]:
CLASS_DICT = dict(zip(('real', 'fake', 'opinion', 'polarized', 'satire', 'promotional', 'correction'),
                      (1, 2, 3, 5, 7, 9, 11)))

In [3]:
headers = r.utils.default_headers()
headers.update({'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/52.0'})

In [4]:
nativead = pd.read_excel("data/Nativead.xlsx")
nativead.head()

,Site,Status,Coder,Story 1,Story 2,Story 3
0,Yahoo News,DONE,DI,https://goingthere.yahoo.com/post/133803688011...,https://goingthere.yahoo.com/post/130087715036...,https://goingthere.yahoo.com/post/130160620511...
1,ABC News \n,Done,DI,http://abcnews.go.com/US/wireStory/providence-...,http://abcnews.go.com/Politics/obama-hits-trum...,http://abcnews.go.com/Politics/justice-antonin...
2,CNN Network,DONE,BWW,http://sponsorcontent.cnn.com/interactive/cine...,http://sponsorcontent.cnn.com/amazon/article/8...,http://sponsorcontent.cnn.com/subaru/article/a...
3,NBC News Digital,DONE,NJE,http://www.nbcnews.com/video/this-designer-cut...,http://www.nbcnews.com/video/baby-boomers-are-...,http://www.nbcnews.com/video/he-creates-furnit...
4,HuffingtonPost.com,DONE,BWW,http://www.huffingtonpost.com/2015/06/04/best-...,http://www.huffingtonpost.com/kathy-magee/a-sm...,http://www.huffingtonpost.com/2015/05/29/evolu...


In [5]:
promo = pd.read_excel("data/PR_content_news.xlsx")
promo.head()

,source,URLs
0,U.S. Department of State,https://www.state.gov/release-of-the-united-st...
1,U.S. Department of State,https://www.state.gov/conclusion-of-negotiatio...
2,U.S. Department of State,https://www.state.gov/u-s-mexico-joint-declara...
3,U.S. Department of State,https://www.state.gov/united-states-restricts-...
4,U.S. Department of State,https://www.state.gov/president-donald-j-trump...


In [6]:
nativead['Status'].value_counts()

DONE                38
Done                 5
N/A, now defunct     1
Found only 1         1
Name: Status, dtype: int64

In [7]:
nativead = nativead.astype(str)
nativead = nativead[nativead['Status'].apply(str.lower)=='done']
nativead.isnull().sum()

Site       0
Status     0
Coder      0
Story 1    0
Story 2    0
Story 3    0
dtype: int64

In [8]:
nativead = nativead.drop(['Status', 'Coder'], axis=1)
nativead.head(n=5)

,Site,Story 1,Story 2,Story 3
0,Yahoo News,https://goingthere.yahoo.com/post/133803688011...,https://goingthere.yahoo.com/post/130087715036...,https://goingthere.yahoo.com/post/130160620511...
1,ABC News \n,http://abcnews.go.com/US/wireStory/providence-...,http://abcnews.go.com/Politics/obama-hits-trum...,http://abcnews.go.com/Politics/justice-antonin...
2,CNN Network,http://sponsorcontent.cnn.com/interactive/cine...,http://sponsorcontent.cnn.com/amazon/article/8...,http://sponsorcontent.cnn.com/subaru/article/a...
3,NBC News Digital,http://www.nbcnews.com/video/this-designer-cut...,http://www.nbcnews.com/video/baby-boomers-are-...,http://www.nbcnews.com/video/he-creates-furnit...
4,HuffingtonPost.com,http://www.huffingtonpost.com/2015/06/04/best-...,http://www.huffingtonpost.com/kathy-magee/a-sm...,http://www.huffingtonpost.com/2015/05/29/evolu...


In [9]:
def get_article_url(urls=pd.DataFrame):
    return np.random.choice(urls.to_numpy().flatten())

story_urls = nativead[['Story 1', 'Story 2', 'Story 3']]
test_url = get_article_url(story_urls)
article = r.get(test_url, headers)
while not article:
    print(test_url + ' Response code 404, trying again...')
    try:
        test_url = get_article_url(story_urls)
        article = r.get(test_url, headers)
    except r.exceptions.MissingSchema:
        continue
else:
    data = article.text
    soup = BeautifulSoup(data)

http://paidpost.nytimes.com/vacheron/the-shape-of-harmony/elegance-of-substance.html Response code 404, trying again...


In [10]:
print("\n".join([para.get_text(strip=True) for para in soup.find_all('p')]))

Desktop notifications are on   |Turn off
Get breaking news alerts fromThe Washington Post
Turn on desktop notifications?
When it comes to the milk America drinks, cows shouldn't get all the credit. Dairy farmers like Dave and Laurie Kyle of Elkhorn, Wis., put in just as much hard work as their bovine counterparts.
Morning for these dairy producers starts early. By 4:30, the Kyles and their farmhands are up for the first of two rounds of milking their 110 registered Holstein cows. While the cows are in the milking parlor, Dave is busy cleaning the barn, replenishing bedding and filling watering units with fresh water. Next, it's time to feed the girls a nutritious breakfast of farm-grown hay, corn and silage (a high-moisture feed that is often made from grass crops that provides many of the nutrients present in plants) mixed with soybean meal and Milwaukee malted rye. (During the warmer months, the cows also graze on fresh pasture.) Dave meets with a local nutritionist weekly, and feed 

In [11]:
sponsored_div = soup.find_all('div', attrs={'class':re.compile("sponsor|provided")})
links = soup.findAll('a', attrs={'href': re.compile("^https*:\/\/")})
#[j.find_all('div', attrs={'class':'item'}) for j in sponsored_div if j.find_all('div', attrs={'class':'item'})!=[]]
[j.get('href') for j in links][0:5]

['https://www.washingtonpost.com/?tid=tst_homelink',
 'https://www.washingtonpost.com/elections/',
 'https://www.washingtonpost.com/opinions/',
 'https://www.washingtonpost.com/national/investigations/',
 'https://www.washingtonpost.com/coronavirus/']

In [12]:
try:
    final_frame = pd.read_csv('data/promotional_text.csv')
except:
    texts = []
    urls = nativead[['Story 1', 'Story 2', 'Story 3']].to_numpy().flatten().tolist() + promo['URLs'].to_list()
    good_urls = []
    more_urls = []
    badresponses = 0
    for i, url in enumerate(urls):
        if url=='nan':
            texts.append('')
            continue
        try:
            article = r.get(url, headers)
            if article:
                good_urls.append(url)
                links = soup.select('body p > a')
                data = article.text
                soup = BeautifulSoup(data)
                article = " ".join([para.get_text(strip=True) for para in soup.find_all('p')])
                texts.append(article)
            else:
                badresponses += 1
                texts.append('')
        except r.exceptions.TooManyRedirects:
            texts.append('')
            continue
        sys.stdout.flush()
        sys.stdout.write("\r" + f"{i!s}/{len(urls)!s} with {badresponses!s} bad responses.")
    sources = nativead['Site'].repeat(3).to_list() + promo['source'].to_list()
    final_frame = pd.DataFrame({'source':sources,
                                'url': urls,
                                'text': texts})
    final_frame = final_frame[final_frame['text']!='']
    final_frame = final_frame[final_frame['text']!=' ']
    final_frame.to_csv('data/promotional_text.csv', index=False)

In [13]:
final_frame.head()

,source,url,text
0,ABC News \r\n,http://abcnews.go.com/Politics/obama-hits-trum..., -- \r\nPresidentObamasaid today that he does...
1,ABC News \r\n,http://abcnews.go.com/Politics/justice-antonin..., -- \r\nEven though the relatives and doctors...
2,CNN Network,http://sponsorcontent.cnn.com/interactive/cine...,“Everything that needs to be said hasalready b...
3,NBC News Digital,http://www.nbcnews.com/video/this-designer-cut...,Watch live: NY Gov. Andrew Cuomo holds coronav...
4,NBC News Digital,http://www.nbcnews.com/video/baby-boomers-are-...,Watch live: NY Gov. Andrew Cuomo holds coronav...


In [14]:
with open('promotional_news_urls.txt', mode='w') as fileout:
    fileout.write("\n".join(final_frame.url.to_list()))

We can now assemble `ArticleVectors` straight-on, supplying one or both of the article text and the URL. This is demonstrated below.

In [15]:
%%timeit
x = fe.ArticleVector(text='sample text', url=test_url)
x.vector

19.1 ms ± 4.81 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [16]:
%%timeit
x = fe.ArticleVector(url=test_url)
x.vector

20.1 s ± 2.57 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


Full disclosure: Due to limitations in the current version of `goose3` I am unable to properly handle `ReadTimeout` exceptions, which occur when an article takes longer than expected for `requests` to receive data from. I can arbitrarily extend the timeout threshold, but this of course means that each article will take even longer to extract. Instead I opt to use the text that I scraped using `BeautifulSoup` and **omit the article title** and Goose extraction altogether.

This is of no consequence in the end -- the title is only used to determine a portion of an article's `all_caps_index` and `apa_index`.

In [17]:
if not os.path.isfile('./data/promotional_news_vectors.txt'):
    vectors = [fe.ArticleVector(text=row['text'], url=row['url']) for _, row in final_frame.iterrows()]

In [18]:
if not os.path.isfile('./data/promotional_news_vectors.txt'):
    with open('data/promotional_news_vectors.txt', mode='w') as fileout:
        fileout.write("\n".join([str(" ".join(list(map(str, j.vector)) + ['9'])) for j in vectors]))

On to the NYTimes BBC Corrections dataset.

I use a Powershell `cmdlet` originally obtained from [this source](http://woshub.com/powershell-get-folder-sizes/) to find those directories whose size is greater than 19.57Kb (the size of an empty Git repository). These then contain text snapshots of the first revisions of articles.

```
gci -force '.' -ErrorAction SilentlyContinue | 
    ? { ($_ -is [io.directoryinfo]) } | % {
        $len = 0
        gci -recurse -force $_.fullname -ErrorAction SilentlyContinue | 
           % { $len += $_.length}
        $_.fullname, ', {0:N2}' -f ($len / 1Mb)
} | Out-File -FilePath './nytbbcdirs.csv' -Encoding UTF8
```

In [19]:
folders = pd.read_csv("data/nytbbcdirs.csv", encoding='utf-8', header=None, names=['dir', 'size'])
folders['subdirs'] = [sum(1 for _ in f) for f in
                          [os.scandir(directory) for directory in folders['dir']]]
folders = folders[folders['size'] > 19.57]

In [20]:
path = folders.sort_values(['size', 'subdirs'], ascending=False).iloc[0]['dir']
#path = path.replace('\\', '/')
path

'C:\\Users\\hunter\\dev\\data\\nytimes_bbc_2017_2020\\2019-01_new'

The folder found above is the most complete archive of the first revisions of corrected articles.

We will [recursively open all bottom-level subfiles](https://stackoverflow.com/a/13617120) as text in the two URL directories. The second line is always article title.

In [21]:
file_set = set()
for dir_, dirs, files in os.walk(path):
    if '.git' in dirs:
        dirs.remove('.git')
    if '.gitignore' in files:
        files.remove('.gitignore')
    for file_name in files:
        rel_dir = os.path.relpath(dir_, path)
        rel_file = os.path.join(rel_dir, file_name)
        file_set.add(rel_file)
file_set = tuple(file_set); file_set[0:5]

('www.bbc.co.uk\\news\\world-latin-america-47053701',
 'www.nytimes.com\\2019\\01\\18\\opinion\\donald-trump-russia-putin.html',
 'www.bbc.co.uk\\news\\world-europe-46798191',
 'www.nytimes.com\\2019\\01\\07\\arts\\music\\21-savage-i-am-i-was-billboard-chart.html',
 'https_\\www.nytimes.com\\2019\\01\\13\\realestate\\homes-that-sold-for-around-750000.html')

In [36]:
url_set = [url.replace('https_', 'https:/') for url in file_set]
url_set = [url.replace('http_', 'http:/') for url in url_set]
url_set = [url.replace('\\', '/') for url in url_set]
url_set[:5]

['www.bbc.co.uk/news/world-latin-america-47053701',
 'www.nytimes.com/2019/01/18/opinion/donald-trump-russia-putin.html',
 'www.bbc.co.uk/news/world-europe-46798191',
 'www.nytimes.com/2019/01/07/arts/music/21-savage-i-am-i-was-billboard-chart.html',
 'https://www.nytimes.com/2019/01/13/realestate/homes-that-sold-for-around-750000.html']

In [37]:
pd.Series(url_set).to_csv('./data/corrections_urls.txt', sep=' ', index=False, header=None)

In [24]:
titles, texts = [], []

for file in file_set:
    with open(os.path.join(path, file), mode='r', encoding = 'utf-8', errors='ignore') as filein:
        text = filein.readlines()
        texts.append(" ".join(text[2:]))
        titles.append(text[1])

In [25]:
corrections = pd.DataFrame({'title':titles,
                            'text':texts})

In [26]:
feats = ["url_ending_index", "from_reputable_source_index",
         "today_index", "grammar_index", "quotation_index",
         "past_tense_index", "present_tense_index", "should_index",
         "opinion_index", "all_caps_index", "from_satire_source_index",
         "exclamation_index", "apa_index", "name_source_index",
         "interjection_index", "you_index", "dot_gov_ending_index",
         "from_unreputable_source_index"]
try:
    corrections_vectors = pd.read_csv('./data/correction_news_vectors.txt', delim_whitespace=True, header=None, names=feats)
except FileNotFoundError:
    corrections_vectors = []

    for i, row in corrections.iterrows():
        sys.stdout.flush()
        sys.stdout.write("\r" + f"{i!s}/{len(file_set)}")
        corrections_vectors.append(fe.ArticleVector(text=row['text'], title=row['title']).vector)

    corrections_vectors = pd.DataFrame(corrections_vectors, columns=feats)
    corrections_vectors = corrections_vectors.assign(url_ending_index=1)
    corrections_vectors = corrections_vectors.assign(dot_gov_ending_index=0)
    corrections_vectors = corrections_vectors.assign(label=11)

    corrections_vectors.to_csv('./data/correction_news_vectors.txt', header=None, index=None, sep=' ')

In [27]:
corrections_vectors.shape

(6805, 18)

In [28]:
d_test = pd.read_csv('./data/human_test_articles.csv')
d_test = d_test.drop('Citizen Journalism', axis=1)
d_test.head()

,Native Ads:,Press Release:,False news,Real news,Opinion,Satire,Misreporting,Polarized
0,https://www.businessinsider.com/sc/how-to-crea...,https://sports.yahoo.com/mango-non-medical-clo...,https://web.archive.org/web/20200601103700/htt...,https://www.cnn.com/2020/05/12/health/us-coron...,https://www.cnn.com/2020/06/12/opinions/corona...,https://babylonbee.com/news/pelosi-rips-up-bible,https://www.nytimes.com/2020/06/08/world/middl...,https://www.nationalreview.com/2020/06/trump-a...
1,https://www.inc.com/verizon/the-new-security-c...,https://www.automotiveworld.com/news-releases/...,https://web.archive.org/web/20200430182051/htt...,https://apnews.com/f18aadd2d6a2868c3765a02c2c9...,https://www.cnn.com/2020/06/11/opinions/time-p...,https://worldnewsdailyreport.com/florida-teen-...,https://www.reuters.com/article/us-health-coro...,https://dailycaller.com/2020/06/19/tucker-carl...
2,https://nationalpost.com/sponsored/food-drink-...,https://thedcline.org/2020/06/11/press-release...,https://web.archive.org/web/20200424225130/htt...,https://www.nytimes.com/2020/05/29/technology/...,https://www.theguardian.com/commentisfree/2019...,https://www.duffelblog.com/2014/12/army-milita...,https://apnews.com/eac82d6b410648ae8cccfb54f91...,https://www.breitbart.com/social-justice/2020/...
3,https://www.forbes.com/sites/qatarfoundation/2...,https://www.prnewswire.com/news-releases/money...,https://archive.is/uGHEb,https://www.bbc.com/news/world-us-canada-52557291,https://www.theguardian.com/commentisfree/2020...,https://web.archive.org/web/20200630111931/htt...,https://www.foxnews.com/us/correction-hot-air-...,https://www.thedailybeast.com/local-businesses...
4,https://thisisreno.com/2020/06/remsa-partners-...,https://www.globenewswire.com/news-release/202...,http://archive.vn/8Kp0r#selection-427.1-427.40,https://popular.info/p/amazon-soliciting-publi...,https://www.usatoday.com/story/opinion/2020/06...,https://www.gishgallop.com/area-skydiver-caugh...,https://spectator.org/not-at-home-again-in-ind...,https://slate.com/news-and-politics/2020/06/wa...


In [29]:
d_test_final = dict(real=d_test['Real news'].dropna(),
                    fake=d_test['False news'].dropna(),
                    opinion=d_test['Opinion'].dropna(),
                    polarized=d_test['Polarized'].dropna(),
                    satire=d_test['Satire'].dropna(),
                    promotional=pd.concat([d_test['Native Ads:'], d_test['Press Release:']]).dropna().reset_index(drop=True),
                    correction=d_test['Misreporting'].dropna())

In [30]:
human_set_vectors = {}
for label, url_series in d_test_final.items():
        human_set_vectors[label] = url_series.apply(lambda j: fe.ArticleVector(url=j).vector)

In [31]:
vectors_labeled = []
for label, vec_series in human_set_vectors.items():
    for vector in vec_series:
        vectors_labeled.append(vector + [CLASS_DICT[label]])

In [33]:
d_human = pd.DataFrame(vectors_labeled, columns=feats+['label'])
d_human.head()

,url_ending_index,from_reputable_source_index,today_index,grammar_index,quotation_index,past_tense_index,present_tense_index,should_index,opinion_index,all_caps_index,from_satire_source_index,exclamation_index,apa_index,name_source_index,interjection_index,you_index,dot_gov_ending_index,from_unreputable_source_index,label
0,1,0,0.000000,0.050505,0.040404,0.053030,0.136364,0.000000,0,0.020202,1,0.0,1,0.073232,0.0,0.000000,0,0,1
1,1,0,0.000000,0.030060,0.004008,0.080160,0.048096,0.004008,0,0.018036,1,0.0,0,0.246493,0.0,0.000000,0,0,1
2,1,0,0.002817,0.014085,0.000000,0.095775,0.076056,0.005634,0,0.000000,1,0.0,0,0.157746,0.0,0.000000,0,0,1
3,1,0,0.000000,0.034568,0.009877,0.093827,0.074074,0.000000,0,0.051852,1,0.0,0,0.167901,0.0,0.000000,0,0,1
4,0,0,0.000000,0.044625,0.022312,0.072008,0.081136,0.002028,0,0.015213,1,0.0,1,0.091278,0.0,0.001014,0,0,1


In [34]:
d_human.to_csv('./data/human_test_vectors.csv', index=False)